In [1]:
from io import BytesIO
import lxml.html
import pandas as pd
import requests

In [2]:
# Constants
REQUEST_URL = "https://bulbapedia.bulbagarden.net/w/api.php?action=parse&format=json&page=List_of_Pok%C3%A9mon_Trading_Card_Game_expansions"

In [3]:
# graceful fetch - error handling to be dealt properly in implementation
def fetch_contents(uri):
    try:
        response = requests.get(uri, timeout=5)
        response.raise_for_status()
    
        # if successful
        return response
    except requests.exceptions.HTTPError as errh:
        print(errh)
    except requests.exceptions.ConnectionError as errc:
        print(errc)
    except requests.exceptions.Timeout as errt:
        print(errt)
    except requests.exceptions.RequestException as err:
        print(err)

In [4]:
# fetch and save to file
response = fetch_contents(REQUEST_URL)
# file_representation = BytesIO(html_content)
with open("temp.txt", "wb") as fd:
    for chunk in response.iter_content(chunk_size=128):
        fd.write(chunk)

In [17]:
# successful experiment: replace the offending colspans, and convert from string to int
with open("temp.txt", "r") as fd:
    line = fd.readline()
    
    print(line.count('colspan=\\"2\\"'))
    new_line = line.replace('colspan=\\"2\\"', "colspan=2")
    print(new_line.count('colspan=\\"2\\"'))

    with open("temp2.txt", "w") as fd:
        fd.write(new_line)

3
0


In [19]:
# debug print
def print_all_tables(file):
    tables = pd.read_html(file)
    
    for index, table in enumerate(tables):
        print("---------------------------------------------------------------\n")
        print(f"Table No. {index}:")
        print(table)

print_all_tables("temp2.txt")

---------------------------------------------------------------

Table No. 0:
  Set no.\n  Symbol\n Logo of Expansion\n Name of Expansion\n  \
0       1\n  \u2014\n            \u2014\n          Base Set\n   
1       2\n        \n                  \n            Jungle\n   
2       3\n        \n                  \n            Fossil\n   
3       4\n        \n                  \n        Base Set 2\n   
4       5\n        \n                  \n       Team Rocket\n   
5       6\n        \n                  \n        Gym Heroes\n   
6       7\n        \n                  \n     Gym Challenge\n   

       Type of Expansion\n      No. of cards\n       Release date\n Set abb.\n  
0  Main Series Expansion\n               102\n    January 9, 1999\n       BS\n  
1  Main Series Expansion\n                64\n      June 16, 1999\n       JU\n  
2  Main Series Expansion\n                62\n   October 10, 1999\n       FO\n  
3  Main Series Expansion\n               130\n  February 24, 2000\n       B2\

In [21]:
# read file post-edit
tables = pd.read_html("temp2.txt")
table = tables[0]
last_table = tables[-1]

In [25]:
# debug prints
if "Set abb.\\n" in table.columns:
    print("found")
else:
    print("not found")
if "Set abb.\\n" in last_table.columns:
    print("found")
else:
    print("not found")

found
not found


In [26]:
# Extract only desired tables
filtered_tables = [df for df in tables if "Set abb.\\n" in table.columns]

In [28]:
# pretty print (debug)
for index, df in enumerate(filtered_tables):
    print("---------------------------------------------------------------\n")
    print(f"Table No. {index}:")
    print(df)

---------------------------------------------------------------

Table No. 0:
  Set no.\n  Symbol\n Logo of Expansion\n Name of Expansion\n  \
0       1\n  \u2014\n            \u2014\n          Base Set\n   
1       2\n        \n                  \n            Jungle\n   
2       3\n        \n                  \n            Fossil\n   
3       4\n        \n                  \n        Base Set 2\n   
4       5\n        \n                  \n       Team Rocket\n   
5       6\n        \n                  \n        Gym Heroes\n   
6       7\n        \n                  \n     Gym Challenge\n   

       Type of Expansion\n      No. of cards\n       Release date\n Set abb.\n  
0  Main Series Expansion\n               102\n    January 9, 1999\n       BS\n  
1  Main Series Expansion\n                64\n      June 16, 1999\n       JU\n  
2  Main Series Expansion\n                62\n   October 10, 1999\n       FO\n  
3  Main Series Expansion\n               130\n  February 24, 2000\n       B2\